In [2]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import numpy as np
from datetime import datetime
import time

In [7]:
pd.read_json('../../features/btc_influencers.json')

,nick
0,@APompliano
1,@ErikVorhees
2,@TyDanielSmith
3,@ToneVays
4,@CryptoCobain
5,@Tyler
6,@VitalikButerin
7,@CryptoWendyO
8,@StackingUSD
9,@Girlgone_Crypto


# Set up

In [8]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAKHzbwEAAAAANrBlqwYokl3ttFARZ4ZmiSD7Bw0%3DRa8THgCyIJMa416ckXp3dLfyC4JgfsugJPgLkRT802r0hynEoy'

In [9]:
def auth():
    return os.getenv('TOKEN')

In [10]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [11]:
# Search user information
def create_user_url(name_list):
    """
    name_list: a string in form of 'xx1,xx2,xx3'
    """
    
    search_url = "https://api.twitter.com/2/users/by" # User information

    #change params based on the endpoint you are using
    query_params = {'usernames': name_list,
                     'user.fields': 'created_at,public_metrics'}
    return (search_url, query_params)

In [6]:
# Search tweets based on keyword and username
def create_keyword_url(keyword, user_name, max_results, start_time, end_time_url):
    """
    keyword: a string in form of 'xx OR xx OR xx'
    user_name: a string in form of 'xxx OR xxx OR xxx'
    max_results: number of return
    """
    
    search_url = "https://api.twitter.com/2/tweets/search/all" # Tweets information

    #change params based on the endpoint you are using
    query_params = {'query': '(' + keyword + ') ('+'from:'+ user_name + ')' + ' -is:reply' + ' lang:en',
                    'start_time': '2021-05-15T00:00:00.000Z',
                    'end_time': '2022-05-15T00:00:00.000Z',
                    'max_results': max_results,
                   'tweet.fields': 'created_at,public_metrics,conversation_id'}

    return (search_url, query_params)

In [7]:
# Search replies of a specific tweet
def create_reply_url(to_user, conversation_id, max_results, start_time, end_time):
    """
    to_user: the user who tweet
    conversation_id: the conversation_id of the tweet
    max_result: number of return
    """
    
    
    search_url = "https://api.twitter.com/2/tweets/search/all" # Tweets information

    #change params based on the endpoint you are using
    query_params = {'query': '(conversation_id:'+ conversation_id + ') ( ' + 'to:' + to_user + ')' ,
                    'max_results': max_results,
                    'start_time': '2021-05-15T00:00:00.000Z',
                    'end_time': '2022-05-16T00:00:00.000Z',              # one day extension of replies
                    'expansions': 'in_reply_to_user_id',
                   'tweet.fields': 'created_at,public_metrics,conversation_id'}
    return (search_url, query_params)

In [8]:
def create_rate_url():
    search_url = "https://api.twitter.com/1.1/application/rate_limit_status.json"
    query_params = {'resources' : 'help,users,search,statuses'}
    return (search_url, query_params)

In [9]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [10]:
def manage_rate_limits(response):
    while True:

        # Get number of requests left with our tokens
        remaining_requests = int(response.headers["x-rate-limit-remaining"])

        # If that number is one, we get the reset-time and wait until then, plus 15 seconds.
        if remaining_requests == 1:
            buffer_wait_time = 15
            resume_time = datetime.fromtimestamp( int(response.headers["x-rate-limit-reset"]) + buffer_wait_time )
            print(f"One request from being rate limited. Waiting on Twitter.\n\tResume Time: {resume_time}")
            diff = (resume_time - datetime.today()).seconds
            time.sleep(diff)
            return
        
        if response.ok:
            return response

# load influencer name and get influencers info

In [14]:
# Opening json file
f = open('btc_influencers.json')
  
# returns JSON object as a dictionary
data = json.load(f)
user_list1 = data[0].get('nick').replace('@','')
user_list2 = data[100].get('nick').replace('@','')
count = 0
for i in data[1:]:
    # can only request for 100 user info at a time
    if count<99:
        user_list1 = user_list1 + ',' + (i.get('nick').replace('@',''))
    if count>100:
        user_list2 = user_list2 + ',' + (i.get('nick').replace('@',''))
    count += 1
# Closing file
f.close()

IndexError: list index out of range

In [12]:
# Get influencers info
bearer_token = auth()
headers = create_headers(bearer_token)
user_url1 = create_user_url(user_list1)
user_response1 = connect_to_endpoint(user_url1[0], headers, user_url1[1])
user_url2 = create_user_url(user_list2)
user_response2 = connect_to_endpoint(user_url2[0], headers, user_url2[1])

NameError: name 'user_list1' is not defined

In [42]:
# Save as csv
df = pd.DataFrame(user_response1['data'])
df = pd.concat([df, pd.DataFrame(user_response2['data'])], ignore_index=True)
followers_count = []
tweet_count = []
for i in range(df.shape[0]):
    followers_count.append(df['public_metrics'][i]['followers_count'])
    tweet_count.append(df['public_metrics'][i]['tweet_count'])
df['followers_count'] = followers_count
df['tweet_count'] = tweet_count
df.drop(columns=['public_metrics'], inplace = True)
df.rename(columns = {'id': 'related_user_id','created_at':'joined_time'}, inplace=True)
df.to_csv('user_data.csv')

In [43]:
df.head()

,name,joined_time,username,related_user_id,followers_count,tweet_count
0,Pomp 🌪,2011-07-20T14:43:36.000Z,APompliano,339061487,1613887,64419
1,Ty Smith,2018-02-09T14:33:52.000Z,TyDanielSmith,961971412528517120,21257,4954
2,Tone Vays,2014-06-20T02:28:08.000Z,ToneVays,2577886615,280067,24946
3,Tyler Winklevoss,2009-03-13T18:31:13.000Z,tyler,24222556,1052335,12439
4,vitalik.eth,2011-05-08T16:03:03.000Z,VitalikButerin,295218901,3874335,17325


# Extract data and store as csv

In [44]:
keywords = 'btc OR BTC OR Btc OR xbt OR XBT OR bitcoin OR bitcoins OR BITCOIN OR Bitcoin'
bearer_token = auth()
headers = create_headers(bearer_token)
output_path='twitter_data.csv'
today = datetime.date.today()
start_time = today - datetime.timedelta(month=1)
end_time = today - datetime.timedelta(days=4)
end_time_url = today

In [ ]:
def extract_data(keywords, username, user_id, start_time, end_time, end_time_url):
    
    # extract tweets
    tweets_url = create_keyword_url(keywords, username, 500, start_time, end_time)
    tweets_response = connect_to_endpoint(tweets_url[0], headers, tweets_url[1])
    tweets_df = pd.DataFrame(tweets_response['data'])
    count = tweets_response['meta']['result_count']

    # if number exceed 500, ask for more response
    if 'next_token' in tweets_response['meta']:
        next_token = tweets_response['meta']['next_token']
        while next_token is not None:
            tweets_url = create_keyword_url(keywords, username, 500, start_time, end_time)
            tweets_response = connect_to_endpoint(tweets_url[0], headers, tweets_url[1],next_token = next_token)
            if 'next_token' in tweets_response['meta']:
                next_token = tweets_response['meta']['next_token']
                count += tweets_response['meta']['result_count']
                tweets_df = pd.concat([tweets_df, pd.DataFrame(tweets_response['data'])], ignore_index=True)
            else: 
                count += tweets_response['meta']['result_count']
                tweets_df = pd.concat([tweets_df, pd.DataFrame(tweets_response['data'])], ignore_index=True)
                break

    print("Finish")

    retweet_count = []
    reply_count = []
    like_count = []
    quote_count = []
    for i in range(tweets_df.shape[0]):
        retweet_count.append(tweets_df['public_metrics'][i]['retweet_count'])
        reply_count.append(tweets_df['public_metrics'][i]['reply_count'])
        like_count.append(tweets_df['public_metrics'][i]['like_count'])
        quote_count.append(tweets_df['public_metrics'][i]['quote_count'])
    tweets_df['retweet_count'] = retweet_count
    tweets_df['reply_count'] = reply_count
    tweets_df['like_count'] = like_count
    tweets_df['quote_count'] = quote_count
    tweets_df.drop(columns=['public_metrics','id'], inplace = True)
    tweets_df['is_reply_to_user'] = 0   # 0 means original tweets, 1 means replies
    tweets_df['related_user_id'] = user_id

    tweets_df.to_csv(output_path, mode='w', header = True, index = False)
    
#----------------------------------------------------------------------------------------------------------------------------------------- 
    # Extract replies for each tweet
    loop = tweets_df.shape[0]
    for i in range(loop):
        print('Present progress: '+ str((i+1)) + ' out of ' + str(loop))

        # check rate limit
        url = create_rate_url()
        status_response = requests.request("GET",url[0], headers = headers, params = url[1])
        if not status_response.ok:
            manage_rate_limits(status_response)

        # get tweet replies
        conversation_id = tweets_df.iloc[i]['conversation_id']
        to_user = tweets_df.iloc[i]['related_user_id']

        replies_url = create_reply_url(to_user, conversation_id, 300, start_time, end_time_url)
        replies_response = connect_to_endpoint(replies_url[0], headers, replies_url[1])
        if 'data' in replies_response:
            replies_df = pd.DataFrame(replies_response['data'])
        else:
            time.sleep(1)
            continue

        likes = []
        for j in range(replies_df.shape[0]):
            likes.append(replies_df['public_metrics'][j]['like_count'])
        replies_df['like_count'] = likes

        replies_df.sort_values(by = 'like_count', ascending=False, inplace=True)
        if replies_df.shape[0] > 10:
            replies_df = replies_df[:10]
        replies_df.drop(columns=['public_metrics','id'], inplace = True)
        replies_df['is_reply_to_user'] = 1   # 0 means original tweets, 1 means replies
        replies_df.rename(columns = {'in_reply_to_user_id': 'related_user_id'}, inplace=True)
        replies_df = replies_df.reindex(columns=tweets_df.columns)
        replies_df.to_csv(output_path, mode='a', index = False, header = False)

        time.sleep(1) # make sure not to touch rate limit

        # combine it with tweets_df
        # tweets_df = pd.concat([tweets_df, replies_df], ignore_index=True)

    # tweets_df['user_name'] = username